## Setting a baseline for the competition: 

https://www.kaggle.com/competitions/playground-series-s3e1/data

#### 1. Setting up the configuration

In [1]:
import os

import numpy as np
import pandas as pd

# Preprocessing
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

# Model Building
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

# Data visualisation
import matplotlib.pyplot as plt

In [2]:
data_root = r'C:\kaggle\playground_series_s3e1\playground-series-s3e1'
train_filepath = os.path.join(data_root, 'train.csv')
test_filepath = os.path.join(data_root, 'test.csv')
sample_path = os.path.join(data_root, 'sample_submission.csv')

train_df = pd.read_csv(train_filepath, index_col='id')
test_df = pd.read_csv(test_filepath, index_col='id')
sample = pd.read_csv(sample_path)

train_df.head(5)

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
id,,,,,,,,,
0,2.3859,15.0,3.827160,1.112100,1280.0,2.486989,34.60,-120.12,0.980
1,3.7188,17.0,6.013373,1.054217,1504.0,3.813084,38.69,-121.22,0.946
2,4.7750,27.0,6.535604,1.103175,1061.0,2.464602,34.71,-120.45,1.576
3,2.4138,16.0,3.350203,0.965432,1255.0,2.089286,32.66,-117.09,1.336
4,3.7500,52.0,4.284404,1.069246,1793.0,1.604790,37.80,-122.41,4.500


Data Validation

In [3]:
# Check for null values
train_df.isna().sum()

MedInc         0
HouseAge       0
AveRooms       0
AveBedrms      0
Population     0
AveOccup       0
Latitude       0
Longitude      0
MedHouseVal    0
dtype: int64

In [4]:
# On initial glance these values all appear to be relatively normal and as expected
train_df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000
mean,3.851029,26.057005,5.163124,1.062204,1660.778919,2.831243,35.570030,-119.554329,2.079751
std,1.803167,12.158221,1.206242,0.096490,1302.469608,2.702413,2.083179,1.974028,1.158571
min,0.499900,2.000000,0.851064,0.500000,3.000000,0.950000,32.550000,-124.350000,0.149990
25%,2.602300,17.000000,4.357522,1.020305,952.000000,2.394495,33.930000,-121.800000,1.208000
50%,3.515600,25.000000,5.068611,1.054545,1383.000000,2.744828,34.190000,-118.450000,1.808000
75%,4.699700,35.000000,5.858597,1.088825,1856.000000,3.125313,37.700000,-118.020000,2.660000
max,15.000100,52.000000,28.837607,5.873181,35682.000000,502.990610,41.950000,-114.550000,5.000010


In [5]:
skewed_cols = ['MedInc', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup']
for col in skewed_cols:
    train_df[col] = np.log(train_df[col])
    test_df[col] = np.log(test_df[col])

In [6]:
train_df.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
count,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000,37137.000000
mean,1.249758,26.057005,1.614762,0.057409,7.225711,1.007768,35.570030,-119.554329,2.079751
std,0.444341,12.158221,0.233688,0.072357,0.611200,0.235420,2.083179,1.974028,1.158571
min,-0.693347,2.000000,-0.161268,-0.693147,1.098612,-0.051293,32.550000,-124.350000,0.149990
25%,0.956396,17.000000,1.471904,0.020101,6.858565,0.873173,33.930000,-121.800000,1.208000
50%,1.257210,25.000000,1.623067,0.053110,7.232010,1.009718,34.190000,-118.450000,1.808000
75%,1.547499,35.000000,1.767910,0.085099,7.526179,1.139535,37.700000,-118.020000,2.660000
max,2.708057,52.000000,3.361680,1.770396,10.482402,6.220572,41.950000,-114.550000,5.000010


Feature scaling

An important step of building a successful Neural Network model is for it's features to be scaled. I have done this on all features within this dataset.

In [7]:
features = ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
# Scale our features 
preprocessor = make_column_transformer((StandardScaler(), features))

In [8]:
# Split the target variable out
X_train = train_df.drop(labels=['MedHouseVal'], axis=1)
y_train = train_df["MedHouseVal"]

# set our test data 
X_test = test_df.copy()

In [9]:
# Transform the features
X_train[features] = preprocessor.fit_transform(X_train)
X_test[features] = preprocessor.transform(X_test)

In [10]:
# Split out our training data into a validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)

(29709, 8)
(29709,)
(7428, 8)
(7428,)


#### 2. Model Building

In [11]:
# Set the architecture for our model
model = keras.Sequential([
    layers.Dense(256, activation='relu', input_shape=[X_train.shape[1]]),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu', input_shape=[X_train.shape[1]]),
    layers.Dense(1),
])

In [12]:
# Compile our model setting an optimizer and loss function
model.compile(
    optimizer='adam',
    loss='mean_squared_error' # we can use mse as the loss function as our keras model doesn't have a root_mean_squared_error loss function 
)

In [13]:
early_stopping = callbacks.EarlyStopping(patience=5, min_delta=0.001, restore_best_weights=True)

In [14]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=32,
    epochs=40, # Set this to a large number as early stopping will stop us once the min_delta isn't being reached
    # verbose=0, # suppress output as we'll plot the curves instead
    callbacks=[early_stopping]
)

By plotting the history of each epoch we can see that we begin to overfit on the training set after around 60 epochs. This is the lowest point of validation loss and we have a gradual increase in validation_loss after that point.

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()))

In [ ]:
model.summary()

In [ ]:
# Set our predictions and ravel them into a 1D array
preds = model.predict(X_test).ravel()
preds

In [ ]:
# Check the number of predictions matches the number of rows in the test dataset
len(preds) == len(X_test.index)

In [ ]:
submission = pd.DataFrame({'id': test_df.index, 
                           'MedHouseVal': preds}).set_index('id')
submission.head()

In [ ]:
submission.to_csv("submission_3.csv", index=True)